In [1]:
%pip install openai


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
client = OpenAI(base_url="http://localhost:1234/v1", api_key="anything")

completion = client.chat.completions.create(
  model="LM Studio Community/Phi-3-mini-4k-instruct-GGUF",
  messages=[
    {"role": "system", "content": "Act as helpful assistant"},
    {"role": "user", "content": "what is logic behind calculus?"}
  ],
  temperature=0.7,
)

print(completion.choices[0].message)

ChatCompletionMessage(content=" Calculus, which can be broadly divided into differential calculus and integral calculus, is the mathematical study of change. The primary motivation for its development was to solve problems related to motion and areas under curves in physics. Here's a simplified overview of the logic behind both branches:\n\n1. Differential Calculus: \nDifferential calculus deals with rates at which quantities change, such as velocity (the rate of change of position) or acceleration (the rate of change of velocity). The foundational concept in differential calculus is the derivative. A derivative represents a slope of a function, describing how one quantity changes concerning another. This was first introduced by Isaac Newton and Gottfried Wilhelm Leibniz during their development of infinitesimal calculus.\n\nThe logic behind finding derivatives involves analyzing small increments or differences between two points on a curve. If we consider a curve f(x) at point x, the 

In [8]:
completion.choices[0].message.content

" Calculus, which can be broadly divided into differential calculus and integral calculus, is the mathematical study of change. The primary motivation for its development was to solve problems related to motion and areas under curves in physics. Here's a simplified overview of the logic behind both branches:\n\n1. Differential Calculus: \nDifferential calculus deals with rates at which quantities change, such as velocity (the rate of change of position) or acceleration (the rate of change of velocity). The foundational concept in differential calculus is the derivative. A derivative represents a slope of a function, describing how one quantity changes concerning another. This was first introduced by Isaac Newton and Gottfried Wilhelm Leibniz during their development of infinitesimal calculus.\n\nThe logic behind finding derivatives involves analyzing small increments or differences between two points on a curve. If we consider a curve f(x) at point x, the derivative is defined as the l

In [1]:
%pip install langchain langchain_community langchain_core


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.chat_models import ChatOllama
from langchain_core.messages import HumanMessage, SystemMessage

llm = ChatOllama(model="llama3.1",temperature=0.2)

messages = [SystemMessage(content= "Act as helpful assistant"), 
            HumanMessage(content ="what is logic behind calculus?")]
model_response = llm.invoke(messages)

In [2]:
model_response.content

"Calculus is a branch of mathematics that deals with the study of continuous change, particularly in the context of functions and limits. The logic behind calculus can be broken down into several key concepts:\n\n1. **Limits**: Calculus begins with the concept of limits, which allows us to understand how a function behaves as its input gets arbitrarily close to a certain value. Limits are used to define the basic operations of calculus, such as differentiation and integration.\n2. **Differentiation**: Differentiation is the process of finding the rate at which a function changes as its input changes. This is done by calculating the derivative of the function, which represents the slope of the tangent line to the graph of the function at a given point.\n3. **Integration**: Integration is the process of finding the area under a curve or the accumulation of a quantity over an interval. This is done by calculating the definite integral of the function, which represents the total amount of 

In [ ]:
# write a function for printing hello world 10 times
def hello_world():
    """
    Prints 'Hello World' 10 times.
    """
    for i in range(10):
        print("Hello World")

In [ ]:
import logging
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import requests
import json
import os
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain import PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
import requests
import openai
from prompt import *

app = FastAPI()

# List of proxies
proxies = [
    "http://192.168.10.42:3128",
    "http://192.168.10.43:3128",
    "http://192.168.10.44:3128",
    "http://192.168.10.45:3128",
    "http://192.168.10.49:3128",
    "http://192.168.10.50:3128",
    "http://192.168.10.51:3128",
    "http://192.168.10.53:3128",
]

# Choose a proxy (e.g., the first one in the list)
proxy = proxies[0]

# Configuration for OpenAI API
openai.api_base = "http://localhost:1234/v1"
openai.api_key = "not-needed"

# Set up environment variables for the proxy
# os.environ["http_proxy"] = proxy
# os.environ["https_proxy"] = proxy
# Set the Google API key
os.environ["GOOGLE_API_KEY"] = "Put your API keys here"

# Initialize the model
google_llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", convert_system_message_to_human=True)

#  mistral:7b-instruct
# Initialize the LLM model
# llm = Ollama(model="llama3:8b-instruct-q6_K", stop=["<|im_end|>\n"])
# llm = ChatOllama(model="mistral:7b-instruct", temperature=0)
# llm = ChatOllama(model="codellama:34b-code-q2_K", temperature=0)
# llm = ChatOllama(model="deepseek-coder:6.7b-instruct", temperature=0)
llm = ChatOllama(model="starcoder2:15b-instruct", temperature=0)

def get_model_response(user_prompt, system_prompt):
    # NOTE: No f string and no whitespace in curly braces
    messages = [
        SystemMessage(
            content=system_prompt
        ),
        HumanMessage(
            content=user_prompt
    )
    ]

    # Modified invoking the model
    chat_model_response = llm.invoke(messages)
    response = chat_model_response.content
    return response

class OllamaRequest(BaseModel):
    model: str
    prompt: str

class PromptRequest(BaseModel):
    system_prompt: str
    user_prompt: str

# Define the request body schema
class MessageRequest(BaseModel):
    system_message: str
    user_message: str

# Define the response schema
class MessageResponse(BaseModel):
    response: str

@app.post("/gemini-api", response_model=MessageResponse)
async def chat_endpoint(request: MessageRequest):
    try:
        # Prepare the messages
        system_message = SystemMessage(content=request.system_message)
        human_message = HumanMessage(content=request.user_message)
        
        # Get the response from the LLM
        response = google_llm([system_message, human_message])
        
        # Return the response content
        return MessageResponse(response=response.content)
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/generate")
def generate_text(request: OllamaRequest):
    url = "http://localhost:11434/api/generate"
    headers = {
        'Content-Type': 'application/json',
    }

    payload = request.dict()
    data = {
        "model": payload['model'],
        "prompt": payload['prompt'],
        "stream": False,
    }
    
    response = requests.post(url, headers=headers, data=json.dumps(data))
    
    if response.status_code == 200:
        response_text  = response.text
        data = json.loads(response_text)
        actual_response = data['response']
        return {"response": actual_response}
    else:
        raise HTTPException(status_code=response.status_code, detail=response.text)

@app.post("/generate-response/")
async def generate_response(prompt_request: PromptRequest):
    system_prompt = prompt_request.system_prompt
    user_prompt = prompt_request.user_prompt
    return {"response": get_model_response(system_prompt, user_prompt)}


@app.post("/llama3-70b-api/")
async def generate_llama_response(prompt_request: PromptRequest):
    system_prompt = prompt_request.system_prompt
    user_prompt = prompt_request.user_prompt
    completion =  openai.ChatCompletion.create(
        model="local-model",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.3,
    )
    return completion.choices[0].message.content

@app.get("/codellama-34b-api/")
async def generate_codellama_response(): #prompt_request: PromptRequest
    # system_prompt = prompt_request.system_prompt
    # user_prompt = prompt_request.user_prompt
    return {"response": get_model_response(system_prompt, user_prompt)}

    

if __name__ == "__main__":
    import uvicorn
    logging.basicConfig(level=logging.DEBUG)
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="debug")